# Test of basic operations 
### rotate, keep simple surfaces, keep only bottom parts

In [4]:
import sys
import os
import glob
import numpy as np
import math
from scipy import ndimage
from soma import aims
import anatomist.notebook as ana
from scipy.spatial.transform import Rotation as R

print(sys.version)

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


### Rotation with numpy / scipy

In [5]:
# This directory contains STS branch crops
crop_dir = '/neurospin/dico/data/deep_folding/data/crops/STS_branches/nearest/original/Lcrops'

In [6]:
##### Takes the first nii.gz file from crop_dir
nii_file = glob.glob(os.path.join(crop_dir, "*.nii.gz"))[0]
print(nii_file)

/neurospin/dico/data/deep_folding/data/crops/STS_branches/nearest/original/Lcrops/615441_normalized.nii.gz


In [7]:
cropped_skeleton = aims.read(nii_file)

The scope below is to visualize the file with anatomist:

In [8]:
a = ana.Anatomist()

global modules: /casa/host/build/share/anatomist-5.1/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module anacontrolmenu
loading module foldsplit
loading module simple_controls
Anatomist started.
loading module histogram
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
all python modules loaded
Anatomist started.


In [9]:
mri = a.loadObject(nii_file)
w1 = a.createWindow('Axial')
w2 = a.createWindow('Coronal')
mri.addInWindows([w1, w2])
w1.moveLinkedCursor([35., 35., 35.])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

AnatomistInteractiveWidget(height=308, layout=Layout(height='auto', width='auto'), width=424)

We will now transform the nii.gz file into a numpy array and visualize it with Anatomist:

In [10]:
arr_skel = np.asarray(cropped_skeleton)
np.unique(arr_skel)

array([ 0, 11, 30, 60, 80], dtype=int16)

In [11]:
block = a.createWindowsBlock(2)
w1 = a.createWindow('Axial', block=block)
w2 = a.createWindow('Axial', block=block)
img = aims.Volume(arr_skel)
img.header()['voxel_size'] = [1, 1, 1]
a_img = a.toAObject(img)
a_img.releaseAppRef()
a_img.setName("toto")
a_img.setChanged()
a_img.notifyObservers()
w1.addObjects(mri)
w2.addObjects([a_img])
w1.moveLinkedCursor([35., 35., 35.])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [12]:
arr_skel.shape

(43, 63, 71, 1)

In [13]:
arr_90 = np.rot90(arr_skel, axes=(0,1))

In [14]:
arr_90.shape

(63, 43, 71, 1)

In [15]:
np.unique(arr_90)

array([ 0, 11, 30, 60, 80], dtype=int16)

In [16]:
block = a.createWindowsBlock(2)
w1 = a.createWindow('Axial', block=block)
w2 = a.createWindow('Axial', block=block)
img_90 = aims.Volume(arr_90)
img_90.header()['voxel_size'] = [1, 1, 1]
a_img_90 = a.toAObject(img_90)
a_img_90.releaseAppRef()
a_img_90.setName("toto")
a_img_90.setChanged()
a_img_90.notifyObservers()
w1.addObjects(mri)
w2.addObjects([a_img_90])
w1.moveLinkedCursor([35., 35., 35.])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [17]:
def rotation_3d(a1, a2, a3):
    '''
    Return matrix 4x4 of rotation along 3 canonic axis based on 3 angles.

    - a1, a2, a3 : rotation angles.
    '''
    c1, s1 = np.cos(a1), np.sin(a1)
    c2, s2 = np.cos(a2), np.sin(a2)
    c3, s3 = np.cos(a3), np.sin(a3)
    m1 = np.matrix([
        [c1, -s1, 0, 0],
        [s1, c1,  0, 0],
        [0,   0,  1, 0],
        [0,   0,  0, 1]])
    m2 = np.matrix([
        [1,  0,   0, 0],
        [0, c2, -s2, 0],
        [0, s2,  c2, 0],
        [0,  0,   0, 1]])
    m3 = np.matrix([
        [c3, 0, -s3, 0],
        [0,  1,   0, 0],
        [s3, 0,  c3, 0],
        [0,  0,   0, 1]])
    return m1 * m2 * m3

In [18]:
rotation = rotation_3d(0., np.pi/8, 0.)
# rotation = rotation_3d(0, 0, 0)
translate = np.array([18., 8., 4., 1.])
translation = rotation_3d(0., 0., 0.)
translation[:, 3] = np.asmatrix(translate).T

print(rotation, "\n\n", translation)

[[ 1.          0.          0.          0.        ]
 [ 0.          0.92387953 -0.38268343  0.        ]
 [ 0.          0.38268343  0.92387953  0.        ]
 [ 0.          0.          0.          1.        ]] 

 [[ 1.  0.  0. 18.]
 [ 0.  1.  0.  8.]
 [ 0.  0.  1.  4.]
 [ 0.  0.  0.  1.]]


In [19]:
tr_rot = aims.AffineTransformation3d(rotation)
tr_trans = aims.AffineTransformation3d(translation)
#tr = aims.AffineTransformation3d(r)
#tr.rotationaroundx(math.pi/16)
#tr.setTranslation((18, 8, 4))
rf = getattr(aims, 'ResamplerFactory_%s' % aims.voxelTypeCode(cropped_skeleton))()
resampler = rf.getResampler(0)


In [20]:
print(cropped_skeleton.header()['voxel_size'])

[1, 1, 1, 1]


In [21]:
cropped_skeleton.shape

(43, 63, 71, 1)

In [22]:
out_vol = aims.Volume(80, 80, 80, 1, dtype=aims.voxelTypeCode(cropped_skeleton))
padded_skeleton = aims.Volume(80, 80, 80, 1, dtype=aims.voxelTypeCode(cropped_skeleton))

In [23]:
out_vol.header()['voxel_size'] = [1., 1., 1.]
padded_skeleton.header()['voxel_size'] = [1., 1., 1.]

In [24]:
resampler.resample(cropped_skeleton, tr_trans, 0, padded_skeleton) 
resampler.resample(padded_skeleton, tr_rot, 0, out_vol) 

In [25]:
out_vol.shape

(80, 80, 80, 1)

In [26]:
padded_skeleton.shape

(80, 80, 80, 1)

In [27]:
out_arr = out_vol.arraydata()
np.unique(out_arr)

array([ 0, 11, 30, 60, 80], dtype=int16)

In [28]:
w1 = a.createWindow('Axial')
w2 = a.createWindow('Axial')
a_out = a.toAObject(out_vol)
a_out.addInWindows(w1)
mri.addInWindows(w2)
w1.moveLinkedCursor([35., 35., 35.])

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [30]:
type(out_vol)

soma.aims.Volume_S16

In [31]:
cropped_skeleton.header()

{ 'volume_dimension' : [ 43, 63, 71, 1 ], 'sizeX' : 43, 'sizeY' : 63, 'sizeZ' : 71, 'sizeT' : 1, 'disk_data_type' : 'S16', 'bits_allocated' : 16, 'data_type' : 'S16', 'scale_factor_applied' : 0, 'possible_data_types' : [ 'S16', 'FLOAT', 'DOUBLE' ], 'cal_min' : 0, 'cal_max' : 0, 'freq_dim' : 0, 'phase_dim' : 0, 'slice_dim' : 0, 'slice_code' : 0, 'slice_start' : 0, 'slice_end' : 0, 'slice_duration' : 0, 'storage_to_memory' : [ -1, 0, 0, 42, 0, -1, 0, 62, 0, 0, -1, 70, 0, 0, 0, 1 ], 'voxel_size' : [ 1, 1, 1, 1 ], 'tr' : 1, 'referentials' : [ 'Scanner-based anatomical coordinates', 'Coordinates aligned to another file or to anatomical truth', 'Coordinates aligned to another file or to anatomical truth', '9d19068e-55ee-b1da-79c6-d3f232b8f490' ], 'transformations' : [ [ -1, 0, 0, -26.5, 0, -1, 0, -15.5, 0, 0, -1, 44, 0, 0, 0, 1 ], [ -1, 0, 0, -27, 0, -1, 0, -33, 0, 0, -1, 62, 0, 0, 0, 1 ], [ -1, 0, 0, -27, 0, -1, 0, -33, 0, 0, -1, 62, 0, 0, 0, 1 ], [ 0.94504177570343, 0.0105236228555441, 0.0

In [32]:
out_vol.header()

{ 'volume_dimension' : [ 80, 80, 80, 1 ], 'sizeX' : 80, 'sizeY' : 80, 'sizeZ' : 80, 'sizeT' : 1, 'voxel_size' : [ 1, 1, 1 ], 'texture_min' : 0, 'texture_max' : 80, 'boundingbox_min' : [ 0, 0, 0, 0 ], 'boundingbox_max' : [ 80, 80, 80, 1 ] }

In [33]:
padded_skeleton.header()

{ 'volume_dimension' : [ 80, 80, 80, 1 ], 'sizeX' : 80, 'sizeY' : 80, 'sizeZ' : 80, 'sizeT' : 1, 'voxel_size' : [ 1, 1, 1 ] }

In [34]:
import skimage
im = skimage.data.astronaut()

In [35]:
np.unique(im)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18